# FindArtists Primary Operation Flow

## Open input file

#### Module

In [2]:
import pandas as pd
from find_artists import cache, logger

class ArtistFileReader:
    def __init__(self, file_path):
        self.file_path = file_path

    def read_file(self):
        try:
            if self.file_path.endswith('.csv'):
                data = pd.read_csv(self.file_path)
            elif self.file_path.endswith('.xlsx'):
                data = pd.read_excel(self.file_path)
            else:
                logger.warning("Unsupported file format.")
            cache['artists'] = data['artist'].tolist()
        except FileNotFoundError:
            logger.warning("File not found.")

ModuleNotFoundError: No module named 'find_artists'

#### Test

In [ ]:
reader = ArtistFileReader('src/find_artists/data/input/names_of_artists.xlsx')
reader.read_file()
artists = cache['artists']
print(artists)